In [11]:
import os
import time
from typing import List
from bs4 import BeautifulSoup as Soup
import numpy as np
import ray
from langchain.document_loaders import ReadTheDocsLoader, BSHTMLLoader
from langchain.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain.embeddings.base import Embeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
import pinecone
import time
import os
import demoConfig

# from tqdm.auto import tqdm
# from uuid import uuid4

# add Pinecone API key from app.pinecone.io
api_key = demoConfig.pinecone_api_key
# set Pinecone environment - find next to API key in console
env = demoConfig.pinecone_env
index_name = demoConfig.pinecone_index_name

pinecone.init(api_key=api_key, environment=env)


db_shards = 8
# ray.init()


In [2]:
url = "https://docs.ray.io/en/latest/"
loader = RecursiveUrlLoader(url=url, max_depth=1, extractor=lambda x: Soup(x, "html.parser").text)
docs = loader.load()
len(docs)

34

In [3]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [5]:
# Create or Reinitialize Pinecone index

import time

index_name = "llama-2-7b-example"

if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)

pinecone.create_index(name=index_name, dimension=384, metric="cosine")
# wait for index to finish initialization
while not pinecone.describe_index(index_name).status["ready"]:
    time.sleep(1)

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
)

In [5]:
st = time.time()
chunks = text_splitter.create_documents(
    [doc.page_content for doc in docs], metadatas=[doc.metadata for doc in docs]
)
et = time.time() - st
print(f"Time taken: {et} seconds. {len(chunks)} chunks generated")

Time taken: 0.18439912796020508 seconds. 4184 chunks generated


In [6]:
results = Pinecone.from_documents(chunks, embeddings, index_name=index_name)

In [7]:
type(results)

langchain.vectorstores.pinecone.Pinecone